# <span style="color:cornflowerblue"><center>Predicción del consumo mensual de agua potable en una red de distribución urbana usando una red neuronal LSTM</center></span>
## <span style="color:cornflowerblue"><center>Diplomado en Ciencia de Datos</center></span>
### <span style="color:cornflowerblue"><center>Universidad Nacional de Colombia Sede Bogotá</center></span>
<span style="color:cornflowerblue">**Estudiante**: L. Felipe Castañeda G.</span>

<span style="color:cornflowerblue">**Asesor**: Álvaro Mauricio Montenegro Díaz.</span>

## <span style="color:cornflowerblue">Contenido</span>

## <span style="color:cornflowerblue">Objetivos</span>
- Crear un modelo de red neuronal LSTM para la predicción mensual del consumo de agua potable que use como datos de entrada una serie temporal.
- Crear un modelo de red neuronal LSTM para la predicción mensual del consumo de agua potable que use como datos de entrada varias series temporales.

## <span style="color:cornflowerblue">1. Importar las librerías</span>

In [1]:
import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.layers import Input, Dense, Dropout, LSTM
print("Versión de Tensorflow: ", tf.__version__)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
# from __future__ import absolute_import, division, print_function, unicode_literals

Versión de Tensorflow:  2.9.1


## <span style="color:cornflowerblue">2. Lectura de los datos</span>

Los datos de consumo son los de una red de distribución de agua potable urbana de un municipio del departamento de Caldas. Son 783.000 observaciones del consumo mensual en metros cúbicos de los usuarios residenciales y no residenciales que van desde julio de 2017 hasta diciembre de 2021.

Los datos de consumo presenan las siguientes características:
- Número de la matrícula.
- Nombre del suscriptor.
- Dirección.
- Servicio: Residencial, comercial, industrial y oficial.
- Estrato: 1, 2, 3 y 4.
- Número del contador.
- Lectura actual.
- Lectura anterior.
- Metros consumidos.
- Mes.
- Año.

Los datos climatológicos corresponden a las siguientes características:
- Año.
- Mes.
- Precipitaciones: Ppt (mm).
- Días soleados: DSL (#).
- Temperaturas máxima, mínima y promedio: TMáx (ºC), TMín (ºC) y TPro (ºC).
- Presiones barométricas máxima, mínima y media: Máx PB (hPa), Mín PB (hPa) y Med PB (hPa).
- Velocidad del viento promedio: VVPro (m/s).
- Humedad relativa máxima, mínima y media: HR Máx (%), HR Mín (%) y HR Med (%).
- Radiación solar máxima y media: RS Máx (W/m2) y RS Med (W/m2).
- Índice de aridez mensual: IAM.

Los datos climatológicos pueden ser descargados en [CEDIAC](https://cdiac.manizales.unal.edu.co/indicadores/public/index).

### <span style="color:cornflowerblue">2.1 Lectura de los datos del consumo de agua potable</span>

In [2]:
raw_consumo = pd.read_excel("..\Data\Consumo\Consumo_2017_2021.xlsx")

KeyboardInterrupt: 

### <span style="color:cornflowerblue">2.2 Lectura de los datos climatológicos</span>

In [ ]:
# Read clima excel file.
raw_clima = pd.read_excel("..\Data\Climatologicos_Mensual\Climatologicos_Mensual_2017_2022.xlsx")

### Realizar copias de los datos leídos.

In [ ]:
# Make a copy
consumo = raw_consumo.copy()

In [ ]:
# Make a copy
clima = raw_clima.copy()

## <span style="color:cornflowerblue">3. Limpieza y organización de los datos</span>

### <span style="color:cornflowerblue">3.1 Limpieza y organización de los datos para el consumo de agua potable</span>

In [ ]:
consumo.info()

Revisar si las caractarísticas útiles en el análisis presentan filas sin datos.

In [ ]:
consumo.isna().sum()

La columna 'NumeroContador'presenta 42 filas sin datos. En este caso, como se analizará el consumo, no se borran esas 42 filas.

Remover aquellas filas en las que el consumo sea menor que cero.

In [ ]:
consumo = consumo.drop(consumo.index[consumo['MetrosConsumidos'] < 0])
consumo.shape

Remover las columnas que no se utilizarán en el análisis.

In [ ]:
consumo.drop(['Matricula',"NombreSuscriptor","Direccion","NumeroContador",'Servicio','Estrato'],
             axis=1, inplace=True)
consumo.columns

Realizar una limpieza en la columna 'Mes'.

In [ ]:
consumo["Mes"] = (consumo["Mes"]
                          # .str.lower()
                          #.str.replace("[^a-záéíóúüñ ]","")
                          # .str.replace(" +"," ")
                          .str.replace('[^\w+]|\d+|\s+|_',' ', regex=True)
                          .str.strip()
                          # .str.title()
                          )
consumo["Mes"].astype("str")
consumo["Mes"].head(12)

Cambiar el nombre del mes por su número correspondiente en la columna 'Mes'.

In [ ]:
cm_dict = {'ENERO':1,'FEBRERO':2,'MARZO':3,'ABRIL':4,'MAYO':5,'JUNIO':6,
          'JULIO':7,'AGOSTO':8,'SEPTIEMBRE':9,'OCTUBRE':10,'NOVIEMBRE':11,'DICIEMBRE':12}
consumo = consumo.replace({'Mes':cm_dict})
consumo.head()

Crear una columna 'day' con valores iguales a '1'.

In [ ]:
consumo["day"] = 1
day_consumo = consumo["day"]
consumo.head()

Cambiar el nombre de las columnas 'Ano' y 'Mes', para crear una columna 'Date' con características de fecha en el formato 'AAAA-MM-DD'.

In [ ]:
consumo.rename(columns = {'Ano':'year','Mes':'month'}, inplace=True)
consumo['Date'] = pd.to_datetime(consumo[["year","month","day"]])
consumo.head()

Borrar las columnas 'year', 'month' y 'day'.

In [ ]:
consumo.drop(["year","month","day"], axis=1,inplace=True) #inplace=True, para que me modifique la BD
consumo.head()

Crear un dataframe llamado 'consumo' con las columnas 'Date', 'MetrosConsumidos', 'LecturaActual' y 'LecturaAnterior'.

In [ ]:
consumo['Date'] = pd.to_datetime(consumo.Date,format='%Y-%m-%d')
consumo = pd.DataFrame(consumo[['Date','MetrosConsumidos','LecturaActual','LecturaAnterior']])
consumo.head()

In [ ]:
consumo.info()

Creación del dataframe 'consumo_mensual', agrupando el dataframe 'consumo' por fecha 'Date'

In [ ]:
consumo_mensual = consumo.groupby('Date',as_index=False).sum()
consumo_mensual.info()

In [ ]:
consumo_mensual.shape

In [ ]:
consumo_mensual.head()

### 3.2 Limpieza y organización de datos climatológicos

In [ ]:
print('Shape of the clima:')
print(clima.shape)

In [ ]:
clima.head()

In [ ]:
clima.info()

Remover aquellas filas cuyas entradas no contengan datos.

In [ ]:
clima.isna().sum()

In [ ]:
clima = clima.dropna()
clima.shape

Remover las columnas que no se utilizarán en el análisis.

In [ ]:
clima.drop(['EB', 'DB', 'EB.1', 'DB.1', 'CI', 'SE', 'ZAM'], axis=1, inplace=True)
clima.columns

In [ ]:
clima.head()

Crear una columna 'day' con valores iguales a '1'.

In [ ]:
clima["day"] = 1
day_clima = clima["day"]
clima.head()

Cambiar el nombre de algunas columnas.

In [ ]:
# Rename some columns from 'clima'
clima.rename(columns={"Año":"year", "Mes": "month", "Ppt (mm)":"Ppt", "DSL (#)": "DSL", 'TMáx (ºC)':'TMax','TMín (ºC)':'TMin','TPro (ºC)':'TPro',
                      'Máx PB (hPa)':'PBMax','Mín PB (hPa)':'PBMin','Med PB (hPa)':'PBMed',
                      'HR Máx (%)':'HRMax','HR Mín (%)':'HRMin',"HR Med (%)":"HRMed",
                      'RS Máx (W/m2)':'RSMax',"RS Med (W/m2)":"RSMed"},inplace=True)
clima.columns

Crear una columna 'Date' con características de fecha en el formato 'AAAA-MM-DD'.

In [ ]:
clima['Date'] = pd.to_datetime(clima[["year","month","day"]])
clima.head()

Crear el dataframe 'clima_mensual' con las columnas de interés.

In [ ]:
clima['Date'] = pd.to_datetime(clima.Date,format='%Y-%m-%d')
clima_mensual = pd.DataFrame(clima[['Date','Ppt', 'DSL', 'TMax', 'TMin', 'TPro', 'PBMax', 'PBMin', 'PBMed',
       'VVPro', 'HRMax', 'HRMin', 'HRMed', 'RSMax', 'RSMed', 'IAM']])
clima_mensual.head()

In [ ]:
clima_mensual.info()

### 3.3 Concatenación de los dataframe 'Consumo' y 'Clima' y creación de 'dataframe'

In [ ]:
dataframe = pd.concat([consumo_mensual.set_index('Date'),clima_mensual.set_index('Date')], axis=1, join='inner').reset_index()
dataframe.head()

In [ ]:
dataframe.info()

Indexar 'dataframe' usando la columna 'Date'.

In [ ]:
# Index 'dataframe' by 'Date'
dataframe = dataframe.set_index('Date')
dataframe.head()

In [ ]:
dataframe.info()

## <span style="color:cornflowerblue">4. Análisis exploratorio inicial</span>

### 4.1 Crear una matriz de correlación usando las columnas del dataframe

In [ ]:
corr_matrix_dataframe = dataframe.corr()
plt.figure(figsize=(15,5))
sns.heatmap(corr_matrix_dataframe, annot=True, linewidths=.25, fmt=".2f")

### 4.2 Seleccionar columnas definitivas

Para la predicción de la serie temporal 'Consumo' solo se utilizarán aquellas características que tengan una correlación menor o igual a $-0.15$, o mayor o igual a $0.15$. Dichas características son:
- 'MetrosConsumidos': Consumo.
- 'LecturaActual': Lectura actual.
- 'TMax': Temperatura máxima.
- 'PBMax': Presión barométrica máxima.
- 'VVPro'. Velocidad del viento promedio.
- 'RSMax': Radiación solar máxima.

Se decidió excluir 'LecturaAnterior' pues así como 'LecturaActual', la primera también es un acumulado mensual del consumo de cada usuario.

In [ ]:
dataframe = pd.DataFrame(dataframe[['MetrosConsumidos','LecturaActual', 'TMax', 'PBMax', 'VVPro', 'RSMax']])

In [ ]:
corr_matrix_dataframe = dataframe.corr()
plt.figure(figsize=(10,3))
sns.heatmap(corr_matrix_dataframe, annot=True, linewidths=.25, fmt=".2f")

### 4.3 Estandarización de las columnas

In [ ]:
scaler = StandardScaler()
  
dataframe_standard_scaled = scaler.fit_transform(dataframe)
dataframe_standard_scaled = pd.DataFrame(dataframe_standard_scaled, columns = ['MetrosConsumidos','LecturaActual', 'TMax', 'PBMax',
                                                                              'VVPro', 'RSMax'])

In [ ]:
dataframe_standard_scaled.describe().transpose()

### 4.4 Visualización del consumo y las demás series de interés 

In [ ]:
plt.figure(figsize=(9,4))
plt.plot(dataframe_standard_scaled['MetrosConsumidos'], label='Consumo')
plt.plot(dataframe_standard_scaled['LecturaActual'], label='Lectura actual')
plt.title("Consumo y lectura actual escalados", size = 10)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(9,4))
plt.plot(dataframe_standard_scaled['MetrosConsumidos'], label='Consumo')
plt.plot(dataframe_standard_scaled['TMax'], label='Temperatura máxima')
plt.title("Consumo y temperatura máxima escalados", size = 10)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(9,4))
plt.plot(dataframe_standard_scaled['MetrosConsumidos'], label='Consumo')
plt.plot(dataframe_standard_scaled['PBMax'], label='Presión barométrica máxima')
plt.title("Consumo y presión barométrica máxima escalados", size = 10)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(9,4))
plt.plot(dataframe_standard_scaled['MetrosConsumidos'], label='Consumo')
plt.plot(dataframe_standard_scaled['VVPro'], label='Velocidad del viento promedio')
plt.title("Consumo y velocidad del viento promedio escalados", size = 10)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(9,4))
plt.plot(dataframe_standard_scaled['MetrosConsumidos'], label='Consumo')
plt.plot(dataframe_standard_scaled['RSMax'], label='Radiación solar máxima')
plt.title("Consumo y radiación solar máxima escalados", size = 10)
plt.legend()
plt.show()

## <span style="color:cornflowerblue">5. Predicción mensual del consumo de agua potable usando una red neuronal LSTM</span>

En Python una red neuronal LSTM se puede crear usando la librería [TensorFlow](https://www.tensorflow.org/) por medio de:

1. [tf.keras.layers.Input()](https://www.tensorflow.org/api_docs/python/tf/keras/Input): se usa para instanciar un tensor de Keras el cual se aumenta con ciertos atributos que permiten construir un modelo, solo conociendo las entradas y las salidas del modelo. El argumento 'shape' de esta función es una tupla de valores enteros.
2. [tf.keras.layers.Dropout()](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout): se usa para aplicar una salida a la respectiva entrada. La capa 'Dropout' establece aleatoriamente valores iguales a cero a las unidades de entrada con una frecuencia que es igual al argumento 'rate' en cada paso durante el proceso de entrenamiento, lo cual ayuda a evitar el sobre ajuste (overfitting). El argumento 'rate' de esta función es un valor flotante entre 0 y 1, y es la fracción de las unidades de entrada a 'abandonar'.
3. [tf.keras.layers.LSTM()](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM): Es la capa de LSTM como tal. El argumento 'units' es un entero positivo que indica la dimensionalidad del espacio de salida.
4. [tf.keras.layers.Dense()](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense): Es la capa densamente conectada de la red neuronal.

### 5.1 Preparación de los datos para el entrenamiento

#### Extracción de las series que se van a utilizar para predecir el consumo

In [ ]:
dataset_consumo = pd.DataFrame(dataframe['MetrosConsumidos'], index=dataframe.index)
dataset_lecturaactual = pd.DataFrame(dataframe['LecturaActual'], index=dataframe.index)
dataset_tmax = pd.DataFrame(dataframe['TMax'], index=dataframe.index)
dataset_pbmax = pd.DataFrame(dataframe['PBMax'], index=dataframe.index)
dataset_vvpro = pd.DataFrame(dataframe['VVPro'], index=dataframe.index)
dataset_rsmax = pd.DataFrame(dataframe['RSMax'], index=dataframe.index)

Revisar el tipo de las series:

In [ ]:
print('dataset_consumo type is: ', type(dataset_consumo))
print('dataset_lecturaactual type is: ', type(dataset_lecturaactual))
print('dataset_tmax type is: ', type(dataset_tmax))
print('dataset_pbmax type is: ', type(dataset_pbmax))
print('dataset_vvpro type is: ', type(dataset_vvpro))
print('dataset_rsmax type is: ', type(dataset_rsmax))

Revisar la forma de las series:

In [ ]:
print('dataset_consumo shape is: ', dataset_consumo.shape)
print('dataset_lecturaactual shape is: ', dataset_lecturaactual.shape)
print('dataset_tmax shape is: ', dataset_tmax.shape)
print('dataset_pbmax shape is: ', dataset_pbmax.shape)
print('dataset_vvpro shape is: ', dataset_vvpro.shape)
print('dataset_rsmax shape is: ', dataset_rsmax.shape)
print('dataframe shape is: ', dataframe.shape)

#### Extracción de los valores de cada una de las series

In [ ]:
dataset_consumo_values = dataset_consumo.values
dataset_lecturaactual_values = dataset_lecturaactual.values
dataset_tmax_values = dataset_tmax.values
dataset_pbmax_values = dataset_pbmax.values
dataset_vvpro_values = dataset_vvpro.values
dataset_rsmax_values = dataset_rsmax.values
dataset_values = dataframe.values

Revisar el tipo de los valores de las series:

In [ ]:
print('dataset_consumo_values type is: ', type(dataset_consumo_values))
print('dataset_lecturaactual_values type is: ', type(dataset_lecturaactual_values))
print('dataset_tmax_values type is: ', type(dataset_tmax_values))
print('dataset_pbmax_values type is: ', type(dataset_pbmax_values))
print('dataset_vvpro_values type is: ', type(dataset_vvpro_values))
print('dataset_rsmax_values type is: ', type(dataset_rsmax_values))
print('dataset_values type is: ', type(dataset_values))

Revisar la forma de los valores de las series.

In [ ]:
print('dataset_consumo_values shape is: ', dataset_consumo_values.shape)
print('dataset_lecturaactual_values shape is: ', dataset_lecturaactual_values.shape)
print('dataset_tmax shape_values is: ', dataset_tmax_values.shape)
print('dataset_pbmax_values shape is: ', dataset_pbmax_values.shape)
print('dataset_vvpro_values shape is: ', dataset_vvpro_values.shape)
print('dataset_rsmax_values shape is: ', dataset_rsmax_values.shape)
print('dataset_values shape is: ', dataset_values.shape)

#### Escalar los valores de las series

Crear el objeto 'scaler':

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))

Escalar las series:

In [ ]:
scaled_consumo = np.squeeze(np.array(scaler.fit_transform(dataset_consumo_values)), axis=1)
scaled_lecturaactual = np.squeeze(np.array(scaler.fit_transform(dataset_lecturaactual_values)), axis=1)
scaled_tmax = np.squeeze(np.array(scaler.fit_transform(dataset_tmax_values)), axis=1)
scaled_pbmax = np.squeeze(np.array(scaler.fit_transform(dataset_pbmax_values)), axis=1)
scaled_vvpro = np.squeeze(np.array(scaler.fit_transform(dataset_vvpro_values)), axis=1)
scaled_rsmax = np.squeeze(np.array(scaler.fit_transform(dataset_rsmax_values)), axis=1)
scaled_dataset = np.squeeze(np.array(scaler.fit_transform(dataset_values)))

Revisar el tipo de las series escaladas:

In [ ]:
print('scaled_consumo type is: ', type(scaled_consumo))
print('scaled_lecturaactual type is: ', type(scaled_lecturaactual))
print('scaled_tmax type is: ', type(scaled_tmax))
print('scaled_pbmax type is: ', type(scaled_pbmax))
print('scaled_vvpro type is: ', type(scaled_vvpro))
print('scaled_rsmax type is: ', type(scaled_rsmax))
print('scaled_dataset type is:', type(scaled_dataset))

Revisar la forma de las series escaladas:

In [ ]:
print('scaled_consumo shape is: ', scaled_consumo.shape)
print('scaled_lecturaactual shape is: ', scaled_lecturaactual.shape)
print('scaled_tmax shape is: ', scaled_tmax.shape)
print('scaled_pbmax shape is: ', scaled_pbmax.shape)
print('scaled_vvpro shape is: ', scaled_vvpro.shape)
print('scaled_rsmax shape is: ', scaled_rsmax.shape)
print('scaled_dataset shape is:', scaled_dataset.shape)

### 5.2 Función para crear los datos de entrenamiento

La siguiente función se usará en cada caso de predicción para crear los datos de entrenamiento:

In [ ]:
def multipaso_data(dataset, target, start_index, end_index, history_size, target_size, single_step=False):
    ''' dataset: conjunto de datos para las secuencias de entrada
        target:  conjunto de datos para las secuencias de salida
        start_index: índice inicial para tomar los datos
        end_index: índice final para tomar los datos. None para tomarlos todos
        history_size: tamaño de la ventana para crear las secuencias
        target_size: número de observaciones que se desean pronosticar
        single_step: 'True' para predecir un valor futuro, 'False' para predecir todos los valores hasta target_size
    '''  
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - history_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i)
        data.append(dataset[indices])

        if single_step:
            labels.append(target[i+target_size])
        else:
            labels.append(target[i:i+target_size])

    return np.array(data), np.array(labels)

### <span style="color:cornflowerblue">5.3 Predicción del consumo usando la serie 'Consumo'</span>

#### Creación de los datos de entrenamiento y validación

Revisar las longuitudes de la serie, de los datos de entrenamiento y de los datos de validación:

In [ ]:
len_data = len(dataset_consumo)
len_train = int(len_data*0.8)
len_test = len_data - len_train
print('Longitud de la serie = Longitud de entrenamiento + Longuitud de validación\n', len_data, ' = ', len_train, ' + ', len_test)

Como se usará el $80\%$ de los datos de la serie para entrenamiento y la longuitud de la misma es muy corta, en este caso para fines prácticos y de aplicación del modelo, se usarán un periodo de predección de 3 meses y una ventana de tiempo de 3 meses. Lo anterior para que la función de creación de los datos de entrenamiento y de validación pueda crear vectores de validación que no sean nulos.

**Nota:** Si se reduce el porcentaje de datos de entrenamiento, se podrían incrementar los periodos de predicción y de ventana de tiempo, pero debido a que la serie temporal es muy corta, dicho incremento sería pequeño.  

In [ ]:
periods_to_predict = 3
window_size = 3
train_data_len = int(len_data*0.8)

X_train, y_train = multipaso_data(
    dataset=scaled_consumo, # Conjunto de datos para las secuencias de entrada
    target=scaled_consumo, # Conjunto de datos para las secuencias de salida
    start_index=0, # Índice inicial de donde empezar a tomar los datos
    end_index=train_data_len, # Índice final para tomar los datos. None para tomarlos todos
    history_size=window_size, # Tamaño de la venytana para crear las secuencias
    target_size=periods_to_predict, # Dentro de cuántas observaciones futuras desea pronosticar
    single_step=False #Predecir solamente un valor futuro (=True), predecir todos los valores hasta target_size(=False)
)

X_test, y_test = multipaso_data(
    dataset=scaled_consumo,
    target=scaled_consumo,
    start_index=train_data_len,
    end_index=None,
    history_size=window_size,
    target_size=periods_to_predict,
    single_step=False
)

Revisar el tipo de los datos de entrenamiento y validación:

In [ ]:
print("X_train type is: ", type(X_train))
print("y_train is: ", type(y_train))
print("X_test is: ", type(X_test))
print("y_test is: ", type(y_test))

Revisar la forma de los datos de entrenamiento y validación:

In [ ]:
print("X_train shape is: ", X_train.shape)
print("y_train shape is: ", y_train.shape)
print("X_test shape is: ", X_test.shape)
print("y_test shape is: ", y_test.shape)

#### Generar una semilla para garantizar reproductividad

In [ ]:
tf.random.set_seed(100)

#### Crear el modelo LSTM

La forma de los datos de entrada:

In [ ]:
input_shape = (X_train.shape[1], 1)
units = 50

Las capas:

In [ ]:
inputs = tf.keras.layers.Input(shape=input_shape, batch_size=None, name=None, dtype=None, sparse=None, tensor=None, ragged=None,
                                       type_spec=None, # **kwargs
                              )
x = tf.keras.layers.Dropout(rate=0.1, noise_shape=None, seed=None, # **kwargs,
            name='Dropout_01')(inputs)
x = tf.keras.layers.LSTM(units=units, name='LSTM_layer')(x)
x = tf.keras.layers.Dropout(0.1, name= 'Dropout_02')(x)
outputs = tf.keras.layers.Dense(periods_to_predict)(x)

Creación del modelo:

In [ ]:
model_consumo_consumo_03 = tf.keras.models.Model(inputs=inputs, outputs=outputs, name='series_LSTM_model')
model_consumo_consumo_03.summary()

Compilación del modelo usando el optimizador Adam y la función de pérdida MSE:

In [ ]:
model_consumo_consumo_03.compile(loss='mean_squared_error', optimizer=Adam(0.001))

Entrenamiento del modelo:

In [ ]:
history = model_consumo_consumo_03.fit(X_train, y_train, batch_size=32, epochs=150, verbose=1, validation_split=0.1, shuffle=False)

Visualización de los 'valores de pérdida de los datos de entrenamiento' y de los 'valores de pérdida de los datos de validación', para 'epoch' consecutivos.

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.title("Valores de pérdida de los datos de entrenamiento y de validación. Escala (0,1)", size = 10)
plt.legend();

In [ ]:
X_test.shape

#### Generación de las predicciones para las muestras de entrada:

In [ ]:
prediction = model_consumo_consumo_03.predict(X_test)

Revisar las formas de los datos de validación y las predicciones:

In [ ]:
print(X_test.shape)
print(prediction.shape)
print(y_test.shape)

#### Visualización de las predicciones

In [ ]:
pred =0
y_train_p = X_test[pred,:]
y_test_p = y_test[pred,:]
y_pred_p = prediction[pred,:]

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(np.arange(0, len(y_train_p)), y_train_p, 'g', marker='.', label="Consumo histórico")
plt.plot(np.arange(len(y_train_p), len(y_train_p) + len(y_test_p)), y_test_p, marker='.', label="Consumo")
plt.plot(np.arange(len(y_train_p), len(y_train_p) + len(y_test_p)), y_pred_p, 'r', marker='.', label="Predicción del consumo")
#plt.ylabel('Valor')
plt.xlabel('Time Step')
plt.title("Predicción del consumo de agua para tres periodos usando la serie 'Consumo'. Escala (0,1)", size = 10)
plt.legend()
plt.show();

In [ ]:
model_consumo_consumo_03.save('../Data/modelo_consumo_consumo_03.h5')

### <span style="color:cornflowerblue">5.4 Predicción del consumo usando la serie 'Lectura actual'</span>

#### Creación de los datos de entrenamiento y validación

Revisar las longuitudes de la serie, de los datos de entrenamiento y de los datos de validación:

In [ ]:
len_data = len(dataset_consumo)
len_train = int(len_data*0.8)
len_test = len_data - len_train
print('Longitud de la serie = Longitud de entrenamiento + Longuitud de validación\n', len_data, ' = ', len_train, ' + ', len_test)

Como se usará el $80\%$ de los datos de la serie para entrenamiento y la longuitud de la misma es muy corta, en este caso para fines prácticos y de aplicación del modelo, se usarán un periodo de predección de 3 meses y una ventana de tiempo de 3 meses. Lo anterior para que la función de creación de los datos de entrenamiento y de validación pueda crear vectores de validación que no sean nulos.

**Nota:** Si se reduce el porcentaje de datos de entrenamiento, se podrían incrementar los periodos de predicción y de ventana de tiempo, pero debido a que la serie temporal es muy corta, dicho incremento sería pequeño.  

In [ ]:
periods_to_predict = 3
window_size = 3
train_data_len = int(len_data*0.8)

X_train, y_train = multipaso_data(
    dataset=scaled_lecturaactual, # Conjunto de datos para las secuencias de entrada
    target=scaled_consumo, # Conjunto de datos para las secuencias de salida
    start_index=0, # Índice inicial de donde empezar a tomar los datos
    end_index=train_data_len, # Índice final para tomar los datos. None para tomarlos todos
    history_size=window_size, # Tamaño de la venytana para crear las secuencias
    target_size=periods_to_predict, # Dentro de cuántas observaciones futuras desea pronosticar
    single_step=False #Predecir solamente un valor futuro (=True), predecir todos los valores hasta target_size(=False)
)

X_test, y_test = multipaso_data(
    dataset=scaled_lecturaactual,
    target=scaled_consumo,
    start_index=train_data_len,
    end_index=None,
    history_size=window_size,
    target_size=periods_to_predict,
    single_step=False
)

Revisar el tipo de los datos de entrenamiento y validación:

In [ ]:
print("X_train type is: ", type(X_train))
print("y_train is: ", type(y_train))
print("X_test is: ", type(X_test))
print("y_test is: ", type(y_test))

Revisar la forma de los datos de entrenamiento y validación:

In [ ]:
print("X_train shape is: ", X_train.shape)
print("y_train shape is: ", y_train.shape)
print("X_test shape is: ", X_test.shape)
print("y_test shape is: ", y_test.shape)

#### Generar una semilla para garantizar reproductividad

In [ ]:
tf.random.set_seed(100)

#### Crear el modelo LSTM

La forma de los datos de entrada:

In [ ]:
input_shape = (X_train.shape[1], 1)
units = 50

Las capas:

In [ ]:
inputs = tf.keras.layers.Input(shape=input_shape, batch_size=None, name=None, dtype=None, sparse=None, tensor=None, ragged=None,
                                       type_spec=None, # **kwargs
                              )
x = tf.keras.layers.Dropout(rate=0.1, noise_shape=None, seed=None, # **kwargs,
            name='Dropout_01')(inputs)
x = tf.keras.layers.LSTM(units=units, name='LSTM_layer')(x)
x = tf.keras.layers.Dropout(0.1, name= 'Dropout_02')(x)
outputs = tf.keras.layers.Dense(periods_to_predict)(x)

Creación del modelo:

In [ ]:
model_consumo_lecturaactual_03 = tf.keras.models.Model(inputs=inputs, outputs=outputs, name='series_LSTM_model')
model_consumo_lecturaactual_03.summary()

Compilación del modelo usando el optimizador Adam y la función de pérdida MSE:

In [ ]:
model_consumo_lecturaactual_03.compile(loss='mean_squared_error', optimizer=Adam(0.001))

Entrenamiento del modelo:

In [ ]:
history = model_consumo_lecturaactual_03.fit(X_train, y_train, batch_size=32, epochs=150, verbose=1, validation_split=0.1, shuffle=False)

Visualización de los 'valores de pérdida de los datos de entrenamiento' y de los 'valores de pérdida de los datos de validación', para 'epoch' consecutivos.

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.title("Valores de pérdida de los datos de entrenamiento y de validación. Escala (0,1)", size = 10)
plt.legend();

In [ ]:
X_test.shape

#### Generación de las predicciones para las muestras de entrada:

In [ ]:
prediction = model_consumo_lecturaactual_03.predict(X_test)

Revisar las formas de los datos de validación y las predicciones:

In [ ]:
print(X_test.shape)
print(prediction.shape)
print(y_test.shape)

#### Visualización de las predicciones

In [ ]:
pred =0
y_train_p = X_test[pred,:]
y_test_p = y_test[pred,:]
y_pred_p = prediction[pred,:]

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(np.arange(0, len(y_train_p)), y_train_p, 'g', marker='.', label="Lectura actual histórica")
plt.plot(np.arange(len(y_train_p), len(y_train_p) + len(y_test_p)), y_test_p, marker='.', label="Consumo")
plt.plot(np.arange(len(y_train_p), len(y_train_p) + len(y_test_p)), y_pred_p, 'r', marker='.', label="Predicción del consumo")
#plt.ylabel('Valor')
plt.xlabel('Time Step')
plt.title("Predicción del consumo de agua para tres periodos usando la serie 'Lectura actual'. Escala (0,1)", size = 10)
plt.legend()
plt.show();

In [ ]:
model_consumo_lecturaactual_03.save('../Data/modelo_consumo_lecturaactual_03.h5')

### <span style="color:cornflowerblue">5.5 Predicción del consumo usando la serie 'Temperatura máxima'</span>

#### Creación de los datos de entrenamiento y validación

Revisar las longuitudes de la serie, de los datos de entrenamiento y de los datos de validación:

In [ ]:
len_data = len(dataset_consumo)
len_train = int(len_data*0.8)
len_test = len_data - len_train
print('Longitud de la serie = Longitud de entrenamiento + Longuitud de validación\n', len_data, ' = ', len_train, ' + ', len_test)

Como se usará el $80\%$ de los datos de la serie para entrenamiento y la longuitud de la misma es muy corta, en este caso para fines prácticos y de aplicación del modelo, se usarán un periodo de predección de 3 meses y una ventana de tiempo de 3 meses. Lo anterior para que la función de creación de los datos de entrenamiento y de validación pueda crear vectores de validación que no sean nulos.

**Nota:** Si se reduce el porcentaje de datos de entrenamiento, se podrían incrementar los periodos de predicción y de ventana de tiempo, pero debido a que la serie temporal es muy corta, dicho incremento sería pequeño.  

In [ ]:
periods_to_predict = 3
window_size = 3
train_data_len = int(len_data*0.8)

X_train, y_train = multipaso_data(
    dataset=scaled_tmax, # Conjunto de datos para las secuencias de entrada
    target=scaled_consumo, # Conjunto de datos para las secuencias de salida
    start_index=0, # Índice inicial de donde empezar a tomar los datos
    end_index=train_data_len, # Índice final para tomar los datos. None para tomarlos todos
    history_size=window_size, # Tamaño de la venytana para crear las secuencias
    target_size=periods_to_predict, # Dentro de cuántas observaciones futuras desea pronosticar
    single_step=False #Predecir solamente un valor futuro (=True), predecir todos los valores hasta target_size(=False)
)

X_test, y_test = multipaso_data(
    dataset=scaled_tmax,
    target=scaled_consumo,
    start_index=train_data_len,
    end_index=None,
    history_size=window_size,
    target_size=periods_to_predict,
    single_step=False
)

Revisar el tipo de los datos de entrenamiento y validación:

In [ ]:
print("X_train type is: ", type(X_train))
print("y_train is: ", type(y_train))
print("X_test is: ", type(X_test))
print("y_test is: ", type(y_test))

Revisar la forma de los datos de entrenamiento y validación:

In [ ]:
print("X_train shape is: ", X_train.shape)
print("y_train shape is: ", y_train.shape)
print("X_test shape is: ", X_test.shape)
print("y_test shape is: ", y_test.shape)

#### Generar una semilla para garantizar reproductividad

In [ ]:
tf.random.set_seed(100)

#### Crear el modelo LSTM

La forma de los datos de entrada:

In [ ]:
input_shape = (X_train.shape[1], 1)
units = 50

Las capas:

In [ ]:
inputs = tf.keras.layers.Input(shape=input_shape, batch_size=None, name=None, dtype=None, sparse=None, tensor=None, ragged=None,
                                       type_spec=None, # **kwargs
                              )
x = tf.keras.layers.Dropout(rate=0.1, noise_shape=None, seed=None, # **kwargs,
            name='Dropout_01')(inputs)
x = tf.keras.layers.LSTM(units=units, name='LSTM_layer')(x)
x = tf.keras.layers.Dropout(0.1, name= 'Dropout_02')(x)
outputs = tf.keras.layers.Dense(periods_to_predict)(x)

Creación del modelo:

In [ ]:
model_consumo_tmax_03 = tf.keras.models.Model(inputs=inputs, outputs=outputs, name='series_LSTM_model')
model_consumo_tmax_03.summary()

Compilación del modelo usando el optimizador Adam y la función de pérdida MSE:

In [ ]:
model_consumo_tmax_03.compile(loss='mean_squared_error', optimizer=Adam(0.001))

Entrenamiento del modelo:

In [ ]:
history = model_consumo_tmax_03.fit(X_train, y_train, batch_size=32, epochs=150, verbose=1, validation_split=0.1, shuffle=False)

Visualización de los 'valores de pérdida de los datos de entrenamiento' y de los 'valores de pérdida de los datos de validación', para 'epoch' consecutivos.

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.title("Valores de pérdida de los datos de entrenamiento y de validación. Escala (0,1)", size = 10)
plt.legend();

In [ ]:
X_test.shape

#### Generación de las predicciones para las muestras de entrada:

In [ ]:
prediction = model_consumo_tmax_03.predict(X_test)

Revisar las formas de los datos de validación y las predicciones:

In [ ]:
print(X_test.shape)
print(prediction.shape)
print(y_test.shape)

#### Visualización de las predicciones

In [ ]:
pred =0
y_train_p = X_test[pred,:]
y_test_p = y_test[pred,:]
y_pred_p = prediction[pred,:]

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(np.arange(0, len(y_train_p)), y_train_p, 'g', marker='.', label="Temperatura máxima histórica")
plt.plot(np.arange(len(y_train_p), len(y_train_p) + len(y_test_p)), y_test_p, marker='.', label="Consumo")
plt.plot(np.arange(len(y_train_p), len(y_train_p) + len(y_test_p)), y_pred_p, 'r', marker='.', label="Predicción del consumo")
#plt.ylabel('Valor')
plt.xlabel('Time Step')
plt.title("Predicción del consumo de agua para tres periodos usando la serie 'Temperatura máxima'. Escala (0,1)", size = 10)
plt.legend()
plt.show();

In [ ]:
model_consumo_tmax_03.save('../Data/modelo_consumo_tmax_03.h5')

### <span style="color:cornflowerblue">5.6 Predicción del consumo usando la serie 'Presión barométrica máxima'</span>

#### Creación de los datos de entrenamiento y validación

Revisar las longuitudes de la serie, de los datos de entrenamiento y de los datos de validación:

In [ ]:
len_data = len(dataset_consumo)
len_train = int(len_data*0.8)
len_test = len_data - len_train
print('Longitud de la serie = Longitud de entrenamiento + Longuitud de validación\n', len_data, ' = ', len_train, ' + ', len_test)

Como se usará el $80\%$ de los datos de la serie para entrenamiento y la longuitud de la misma es muy corta, en este caso para fines prácticos y de aplicación del modelo, se usarán un periodo de predección de 3 meses y una ventana de tiempo de 3 meses. Lo anterior para que la función de creación de los datos de entrenamiento y de validación pueda crear vectores de validación que no sean nulos.

**Nota:** Si se reduce el porcentaje de datos de entrenamiento, se podrían incrementar los periodos de predicción y de ventana de tiempo, pero debido a que la serie temporal es muy corta, dicho incremento sería pequeño.  

In [ ]:
periods_to_predict = 3
window_size = 3
train_data_len = int(len_data*0.8)

X_train, y_train = multipaso_data(
    dataset=scaled_pbmax, # Conjunto de datos para las secuencias de entrada
    target=scaled_consumo, # Conjunto de datos para las secuencias de salida
    start_index=0, # Índice inicial de donde empezar a tomar los datos
    end_index=train_data_len, # Índice final para tomar los datos. None para tomarlos todos
    history_size=window_size, # Tamaño de la venytana para crear las secuencias
    target_size=periods_to_predict, # Dentro de cuántas observaciones futuras desea pronosticar
    single_step=False #Predecir solamente un valor futuro (=True), predecir todos los valores hasta target_size(=False)
)

X_test, y_test = multipaso_data(
    dataset=scaled_pbmax,
    target=scaled_consumo,
    start_index=train_data_len,
    end_index=None,
    history_size=window_size,
    target_size=periods_to_predict,
    single_step=False
)

Revisar el tipo de los datos de entrenamiento y validación:

In [ ]:
print("X_train type is: ", type(X_train))
print("y_train is: ", type(y_train))
print("X_test is: ", type(X_test))
print("y_test is: ", type(y_test))

Revisar la forma de los datos de entrenamiento y validación:

In [ ]:
print("X_train shape is: ", X_train.shape)
print("y_train shape is: ", y_train.shape)
print("X_test shape is: ", X_test.shape)
print("y_test shape is: ", y_test.shape)

#### Generar una semilla para garantizar reproductividad

In [ ]:
tf.random.set_seed(100)

#### Crear el modelo LSTM

La forma de los datos de entrada:

In [ ]:
input_shape = (X_train.shape[1], 1)
units = 50

Las capas:

In [ ]:
inputs = tf.keras.layers.Input(shape=input_shape, batch_size=None, name=None, dtype=None, sparse=None, tensor=None, ragged=None,
                                       type_spec=None, # **kwargs
                              )
x = tf.keras.layers.Dropout(rate=0.1, noise_shape=None, seed=None, # **kwargs,
            name='Dropout_01')(inputs)
x = tf.keras.layers.LSTM(units=units, name='LSTM_layer')(x)
x = tf.keras.layers.Dropout(0.1, name= 'Dropout_02')(x)
outputs = tf.keras.layers.Dense(periods_to_predict)(x)

Creación del modelo:

In [ ]:
model_consumo_pbmax_03 = tf.keras.models.Model(inputs=inputs, outputs=outputs, name='series_LSTM_model')
model_consumo_pbmax_03.summary()

Compilación del modelo usando el optimizador Adam y la función de pérdida MSE:

In [ ]:
model_consumo_pbmax_03.compile(loss='mean_squared_error', optimizer=Adam(0.001))

Entrenamiento del modelo:

In [ ]:
history = model_consumo_pbmax_03.fit(X_train, y_train, batch_size=32, epochs=150, verbose=1, validation_split=0.1, shuffle=False)

Visualización de los 'valores de pérdida de los datos de entrenamiento' y de los 'valores de pérdida de los datos de validación', para 'epoch' consecutivos.

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.title("Valores de pérdida de los datos de entrenamiento y de validación. Escala (0,1)", size = 10)
plt.legend();

In [ ]:
X_test.shape

#### Generación de las predicciones para las muestras de entrada:

In [ ]:
prediction = model_consumo_pbmax_03.predict(X_test)

Revisar las formas de los datos de validación y las predicciones:

In [ ]:
print(X_test.shape)
print(prediction.shape)
print(y_test.shape)

#### Visualización de las predicciones

In [ ]:
pred =0
y_train_p = X_test[pred,:]
y_test_p = y_test[pred,:]
y_pred_p = prediction[pred,:]

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(np.arange(0, len(y_train_p)), y_train_p, 'g', marker='.', label="Presión barométrica máxima histórica")
plt.plot(np.arange(len(y_train_p), len(y_train_p) + len(y_test_p)), y_test_p, marker='.', label="Consumo")
plt.plot(np.arange(len(y_train_p), len(y_train_p) + len(y_test_p)), y_pred_p, 'r', marker='.', label="Predicción del consumo")
#plt.ylabel('Valor')
plt.xlabel('Time Step')
plt.title("Predicción del consumo de agua para tres periodos usando la serie 'Presión barométrica máxima'. Escala (0,1)", size = 10)
plt.legend()
plt.show();

In [ ]:
model_consumo_pbmax_03.save('../Data/modelo_consumo_pbmax_03.h5')

### <span style="color:cornflowerblue">5.7 Predicción del consumo usando la serie 'Velocidad del viento promedio'</span>

#### Creación de los datos de entrenamiento y validación

Revisar las longuitudes de la serie, de los datos de entrenamiento y de los datos de validación:

In [ ]:
len_data = len(dataset_consumo)
len_train = int(len_data*0.8)
len_test = len_data - len_train
print('Longitud de la serie = Longitud de entrenamiento + Longuitud de validación\n', len_data, ' = ', len_train, ' + ', len_test)

Como se usará el $80\%$ de los datos de la serie para entrenamiento y la longuitud de la misma es muy corta, en este caso para fines prácticos y de aplicación del modelo, se usarán un periodo de predección de 3 meses y una ventana de tiempo de 3 meses. Lo anterior para que la función de creación de los datos de entrenamiento y de validación pueda crear vectores de validación que no sean nulos.

**Nota:** Si se reduce el porcentaje de datos de entrenamiento, se podrían incrementar los periodos de predicción y de ventana de tiempo, pero debido a que la serie temporal es muy corta, dicho incremento sería pequeño.  

In [ ]:
periods_to_predict = 3
window_size = 3
train_data_len = int(len_data*0.8)

X_train, y_train = multipaso_data(
    dataset=scaled_vvpro, # Conjunto de datos para las secuencias de entrada
    target=scaled_consumo, # Conjunto de datos para las secuencias de salida
    start_index=0, # Índice inicial de donde empezar a tomar los datos
    end_index=train_data_len, # Índice final para tomar los datos. None para tomarlos todos
    history_size=window_size, # Tamaño de la venytana para crear las secuencias
    target_size=periods_to_predict, # Dentro de cuántas observaciones futuras desea pronosticar
    single_step=False #Predecir solamente un valor futuro (=True), predecir todos los valores hasta target_size(=False)
)

X_test, y_test = multipaso_data(
    dataset=scaled_vvpro,
    target=scaled_consumo,
    start_index=train_data_len,
    end_index=None,
    history_size=window_size,
    target_size=periods_to_predict,
    single_step=False
)

Revisar el tipo de los datos de entrenamiento y validación:

In [ ]:
print("X_train type is: ", type(X_train))
print("y_train is: ", type(y_train))
print("X_test is: ", type(X_test))
print("y_test is: ", type(y_test))

Revisar la forma de los datos de entrenamiento y validación:

In [ ]:
print("X_train shape is: ", X_train.shape)
print("y_train shape is: ", y_train.shape)
print("X_test shape is: ", X_test.shape)
print("y_test shape is: ", y_test.shape)

#### Generar una semilla para garantizar reproductividad

In [ ]:
tf.random.set_seed(100)

#### Crear el modelo LSTM

La forma de los datos de entrada:

In [ ]:
input_shape = (X_train.shape[1], 1)
units = 50

Las capas:

In [ ]:
inputs = tf.keras.layers.Input(shape=input_shape, batch_size=None, name=None, dtype=None, sparse=None, tensor=None, ragged=None,
                                       type_spec=None, # **kwargs
                              )
x = tf.keras.layers.Dropout(rate=0.1, noise_shape=None, seed=None, # **kwargs,
            name='Dropout_01')(inputs)
x = tf.keras.layers.LSTM(units=units, name='LSTM_layer')(x)
x = tf.keras.layers.Dropout(0.1, name= 'Dropout_02')(x)
outputs = tf.keras.layers.Dense(periods_to_predict)(x)

Creación del modelo:

In [ ]:
model_consumo_vvpro_03 = tf.keras.models.Model(inputs=inputs, outputs=outputs, name='series_LSTM_model')
model_consumo_vvpro_03.summary()

Compilación del modelo usando el optimizador Adam y la función de pérdida MSE:

In [ ]:
model_consumo_vvpro_03.compile(loss='mean_squared_error', optimizer=Adam(0.001))

Entrenamiento del modelo:

In [ ]:
history = model_consumo_vvpro_03.fit(X_train, y_train, batch_size=32, epochs=150, verbose=1, validation_split=0.1, shuffle=False)

Visualización de los 'valores de pérdida de los datos de entrenamiento' y de los 'valores de pérdida de los datos de validación', para 'epoch' consecutivos.

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.title("Valores de pérdida de los datos de entrenamiento y de validación. Escala (0,1)", size = 10)
plt.legend();

In [ ]:
X_test.shape

#### Generación de las predicciones para las muestras de entrada:

In [ ]:
prediction = model_consumo_vvpro_03.predict(X_test)

Revisar las formas de los datos de validación y las predicciones:

In [ ]:
print(X_test.shape)
print(prediction.shape)
print(y_test.shape)

#### Visualización de las predicciones

In [ ]:
pred =0
y_train_p = X_test[pred,:]
y_test_p = y_test[pred,:]
y_pred_p = prediction[pred,:]

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(np.arange(0, len(y_train_p)), y_train_p, 'g', marker='.', label="Velocidad del viento promedio histórica")
plt.plot(np.arange(len(y_train_p), len(y_train_p) + len(y_test_p)), y_test_p, marker='.', label="Consumo")
plt.plot(np.arange(len(y_train_p), len(y_train_p) + len(y_test_p)), y_pred_p, 'r', marker='.', label="Predicción del consumo")
#plt.ylabel('Valor')
plt.xlabel('Time Step')
plt.title("Predicción del consumo de agua para tres periodos usando la serie 'Velocidad del viento promedio'. Escala (0,1)", size = 10)
plt.legend()
plt.show();

In [ ]:
model_consumo_vvpro_03.save('../Data/modelo_consumo_vvpro_03.h5')

### <span style="color:cornflowerblue">5.8 Predicción del consumo usando la serie 'Radiación solar máxima'</span>

#### Creación de los datos de entrenamiento y validación

Revisar las longuitudes de la serie, de los datos de entrenamiento y de los datos de validación:

In [ ]:
len_data = len(dataset_consumo)
len_train = int(len_data*0.8)
len_test = len_data - len_train
print('Longitud de la serie = Longitud de entrenamiento + Longuitud de validación\n', len_data, ' = ', len_train, ' + ', len_test)

Como se usará el $80\%$ de los datos de la serie para entrenamiento y la longuitud de la misma es muy corta, en este caso para fines prácticos y de aplicación del modelo, se usarán un periodo de predección de 3 meses y una ventana de tiempo de 3 meses. Lo anterior para que la función de creación de los datos de entrenamiento y de validación pueda crear vectores de validación que no sean nulos.

**Nota:** Si se reduce el porcentaje de datos de entrenamiento, se podrían incrementar los periodos de predicción y de ventana de tiempo, pero debido a que la serie temporal es muy corta, dicho incremento sería pequeño.  

In [ ]:
periods_to_predict = 3
window_size = 3
train_data_len = int(len_data*0.8)

X_train, y_train = multipaso_data(
    dataset=scaled_rsmax, # Conjunto de datos para las secuencias de entrada
    target=scaled_consumo, # Conjunto de datos para las secuencias de salida
    start_index=0, # Índice inicial de donde empezar a tomar los datos
    end_index=train_data_len, # Índice final para tomar los datos. None para tomarlos todos
    history_size=window_size, # Tamaño de la venytana para crear las secuencias
    target_size=periods_to_predict, # Dentro de cuántas observaciones futuras desea pronosticar
    single_step=False #Predecir solamente un valor futuro (=True), predecir todos los valores hasta target_size(=False)
)

X_test, y_test = multipaso_data(
    dataset=scaled_rsmax,
    target=scaled_consumo,
    start_index=train_data_len,
    end_index=None,
    history_size=window_size,
    target_size=periods_to_predict,
    single_step=False
)

Revisar el tipo de los datos de entrenamiento y validación:

In [ ]:
print("X_train type is: ", type(X_train))
print("y_train is: ", type(y_train))
print("X_test is: ", type(X_test))
print("y_test is: ", type(y_test))

Revisar la forma de los datos de entrenamiento y validación:

In [ ]:
print("X_train shape is: ", X_train.shape)
print("y_train shape is: ", y_train.shape)
print("X_test shape is: ", X_test.shape)
print("y_test shape is: ", y_test.shape)

#### Generar una semilla para garantizar reproductividad

In [ ]:
tf.random.set_seed(100)

#### Crear el modelo LSTM

La forma de los datos de entrada:

In [ ]:
input_shape = (X_train.shape[1], 1)
units = 50

Las capas:

In [ ]:
inputs = tf.keras.layers.Input(shape=input_shape, batch_size=None, name=None, dtype=None, sparse=None, tensor=None, ragged=None,
                                       type_spec=None, # **kwargs
                              )
x = tf.keras.layers.Dropout(rate=0.1, noise_shape=None, seed=None, # **kwargs,
            name='Dropout_01')(inputs)
x = tf.keras.layers.LSTM(units=units, name='LSTM_layer')(x)
x = tf.keras.layers.Dropout(0.1, name= 'Dropout_02')(x)
outputs = tf.keras.layers.Dense(periods_to_predict)(x)

Creación del modelo:

In [ ]:
model_consumo_rsmax_03 = tf.keras.models.Model(inputs=inputs, outputs=outputs, name='series_LSTM_model')
model_consumo_rsmax_03.summary()

Compilación del modelo usando el optimizador Adam y la función de pérdida MSE:

In [ ]:
model_consumo_rsmax_03.compile(loss='mean_squared_error', optimizer=Adam(0.001))

Entrenamiento del modelo:

In [ ]:
history = model_consumo_rsmax_03.fit(X_train, y_train, batch_size=32, epochs=150, verbose=1, validation_split=0.1, shuffle=False)

Visualización de los 'valores de pérdida de los datos de entrenamiento' y de los 'valores de pérdida de los datos de validación', para 'epoch' consecutivos.

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.title("Valores de pérdida de los datos de entrenamiento y de validación. Escala (0,1)", size = 10)
plt.legend();

In [ ]:
X_test.shape

#### Generación de las predicciones para las muestras de entrada:

In [ ]:
prediction = model_consumo_rsmax_03.predict(X_test)

Revisar las formas de los datos de validación y las predicciones:

In [ ]:
print(X_test.shape)
print(prediction.shape)
print(y_test.shape)

#### Visualización de las predicciones

In [ ]:
pred =0
y_train_p = X_test[pred,:]
y_test_p = y_test[pred,:]
y_pred_p = prediction[pred,:]

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(np.arange(0, len(y_train_p)), y_train_p, 'g', marker='.', label="Radiación solar máxima histórica")
plt.plot(np.arange(len(y_train_p), len(y_train_p) + len(y_test_p)), y_test_p, marker='.', label="Consumo")
plt.plot(np.arange(len(y_train_p), len(y_train_p) + len(y_test_p)), y_pred_p, 'r', marker='.', label="Predicción del consumo")
#plt.ylabel('Valor')
plt.xlabel('Time Step')
plt.title("Predicción del consumo de agua para tres periodos usando la serie 'Radiación solar máxima'. Escala (0,1)", size = 10)
plt.legend()
plt.show();

In [ ]:
model_consumo_rsmax_03.save('../Data/modelo_consumo_rsmax_03.h5')

### <span style="color:cornflowerblue">5.9 Predicción del consumo usando todas las series'</span>

#### Creación de los datos de entrenamiento y validación

Revisar las longuitudes de la serie, de los datos de entrenamiento y de los datos de validación:

In [ ]:
len_data = len(dataset_consumo)
len_train = int(len_data*0.8)
len_test = len_data - len_train
print('Longitud de la serie = Longitud de entrenamiento + Longuitud de validación\n', len_data, ' = ', len_train, ' + ', len_test)

Como se usará el $80\%$ de los datos de la serie para entrenamiento y la longuitud de la misma es muy corta, en este caso para fines prácticos y de aplicación del modelo, se usarán un periodo de predección de 3 meses y una ventana de tiempo de 3 meses. Lo anterior para que la función de creación de los datos de entrenamiento y de validación pueda crear vectores de validación que no sean nulos.

**Nota:** Si se reduce el porcentaje de datos de entrenamiento, se podrían incrementar los periodos de predicción y de ventana de tiempo, pero debido a que la serie temporal es muy corta, dicho incremento sería pequeño.  

In [ ]:
periods_to_predict = 3
window_size = 3
train_data_len = int(len_data*0.8)

X_train, y_train = multipaso_data(
    dataset=scaled_dataset, # Conjunto de datos para las secuencias de entrada
    target=scaled_consumo, # Conjunto de datos para las secuencias de salida
    start_index=0, # Índice inicial de donde empezar a tomar los datos
    end_index=train_data_len, # Índice final para tomar los datos. None para tomarlos todos
    history_size=window_size, # Tamaño de la venytana para crear las secuencias
    target_size=periods_to_predict, # Dentro de cuántas observaciones futuras desea pronosticar
    single_step=False #Predecir solamente un valor futuro (=True), predecir todos los valores hasta target_size(=False)
)

X_test, y_test = multipaso_data(
    dataset=scaled_dataset,
    target=scaled_consumo,
    start_index=train_data_len,
    end_index=None,
    history_size=window_size,
    target_size=periods_to_predict,
    single_step=False
)

Revisar el tipo de los datos de entrenamiento y validación:

In [ ]:
print("X_train type is: ", type(X_train))
print("y_train is: ", type(y_train))
print("X_test is: ", type(X_test))
print("y_test is: ", type(y_test))

Revisar la forma de los datos de entrenamiento y validación:

In [ ]:
print("X_train shape is: ", X_train.shape)
print("y_train shape is: ", y_train.shape)
print("X_test shape is: ", X_test.shape)
print("y_test shape is: ", y_test.shape)

#### Generar una semilla para garantizar reproductividad

In [ ]:
tf.random.set_seed(100)

#### Crear el modelo LSTM

La forma de los datos de entrada:

In [ ]:
input_shape = (X_train.shape[1], 6)
units = 50

Las capas:

In [ ]:
inputs = tf.keras.layers.Input(shape=input_shape, batch_size=None, name=None, dtype=None, sparse=None, tensor=None, ragged=None,
                                       type_spec=None, # **kwargs
                              )
x = tf.keras.layers.Dropout(rate=0.1, noise_shape=None, seed=None, # **kwargs,
            name='Dropout_01')(inputs)
x = tf.keras.layers.LSTM(units=units, name='LSTM_layer')(x)
x = tf.keras.layers.Dropout(0.1, name= 'Dropout_02')(x)
outputs = tf.keras.layers.Dense(periods_to_predict)(x)

Creación del modelo:

In [ ]:
model_consumo_series_03 = tf.keras.models.Model(inputs=inputs, outputs=outputs, name='series_LSTM_model')
model_consumo_series_03.summary()

Compilación del modelo usando el optimizador Adam y la función de pérdida MSE:

In [ ]:
model_consumo_series_03.compile(loss='mean_squared_error', optimizer=Adam(0.001))

Entrenamiento del modelo:

In [ ]:
history = model_consumo_series_03.fit(X_train, y_train, batch_size=32, epochs=150, verbose=1, validation_split=0.1, shuffle=False)

Visualización de los 'valores de pérdida de los datos de entrenamiento' y de los 'valores de pérdida de los datos de validación', para 'epoch' consecutivos.

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.title("Valores de pérdida de los datos de entrenamiento y de validación. Escala (0,1)", size = 10)
plt.legend();

In [ ]:
X_test.shape

#### Generación de las predicciones para las muestras de entrada:

In [ ]:
prediction = model_consumo_series_03.predict(X_test)

Revisar las formas de los datos de validación y las predicciones:

In [ ]:
print(X_test.shape)
print(prediction.shape)
print(y_test.shape)

#### Visualización de las predicciones

In [ ]:
pred =0
y_train_p = X_test[pred,:]
y_test_p = y_test[pred,:]
y_pred_p = prediction[pred,:]

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(np.arange(0, len(y_train_p)), y_train_p, 'g', marker='.') #, label="Radiación solar máxima histórica")
plt.plot(np.arange(len(y_train_p), len(y_train_p) + len(y_test_p)), y_test_p, marker='.', label="Consumo")
plt.plot(np.arange(len(y_train_p), len(y_train_p) + len(y_test_p)), y_pred_p, 'r', marker='.', label="Predicción del consumo")
#plt.ylabel('Valor')
plt.xlabel('Time Step')
plt.title("Predicción del consumo de agua para tres periodos usando todas las series. Escala (0,1)", size = 10)
plt.legend()
plt.show();

In [ ]:
model_consumo_series_03.save('../Data/modelo_consumo_series_03.h5')